<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH19_06_BERT_%EA%B8%B0%EB%B0%98_%EB%B3%B5%ED%95%A9%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8(CTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

복합토픽모델(Combined Topic Models, CTM)

* 빈도수 기반 문서 벡터화 방식인 Bag of Words와 사전 훈련된 언어모델의 문서 임베딩인 SBERT를 결합한 방법


In [1]:
!pip install contextualized-topic-models==2.2.0
#!pip install -U contextualized_topic_models
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.1/785.1 kB 31.5 MB/s eta 0:00:00
  Attempting uninstall: ipython
    Found existing installation: ipython 8.10.0
    Uninstalling ipython-8.10.0:
      Successfully uninstalled ipython-8.10.0
  Attempting uninstall: contextualized-topic-models
    Found existing installation: contextualized-topic-models 2.5.0
    Uninstalling contextualized-topic-models-2.5.0:
      Successfully uninstalled contextualized-topic-models-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 7.16.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 1. 문맥을 반영한 토픽모델(Contextualized Topic Models)
-----------------
문맥을 반영한 BERT의 문서 임베딩 표현력과 기존 토픽 모델의 비지도 학습 능력을 결합하여 문서에서 주제를 가져오는 토픽모델을 말함


### 2. 데이터 로드
-----------
학습을 위한 데이터로 하나의 라인에 하나의 문서로 구성된 파일이 필요함. 영문 위키피디아 파일 로드함

In [2]:
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

--2023-05-27 15:04:56--  https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6208417 (5.9M) [text/plain]
Saving to: ‘dbpedia_sample_abstract_20k_unprep.txt.1’

dbpedia_sample_abst 100%[===================>]   5.92M  --.-KB/s    in 0.04s   

2023-05-27 15:04:56 (144 MB/s) - ‘dbpedia_sample_abstract_20k_unprep.txt.1’ saved [6208417/6208417]



In [3]:
# 상위 3개의 라인만 출력
!head -n 3 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and
Henry Howard, 13th Earl of Suffolk, 6th Earl of Berkshire (8 August 1779 – 10 August 1779) was a British peer, the son of Henry Howard, 12th Earl of Suffolk. His father died on 7 March 1779, leaving behind his pregnant widow. The Earldom of Suffolk became dormant until she


In [4]:
# 파일 저장
text_file = "dbpedia_sample_abstract_20k_unprep.txt"

### 3. 전처리

In [5]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

In [6]:
# 특수 문자가 붙어 동일한 단어가 다른 단어로 인식되지 않도록 특수문자를 제거해주고
# 불용어 제거 

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
documents = [line.strip() for line in open(text_file, encoding='utf-8').readlines()]
sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab, _ = sp.preprocess()

AttributeError: ignored

In [ ]:
# 전처리 결과
preprocessed_documents[:2]

In [ ]:
# 전체 단어 집합 크기
print(len(vocab))

전처리 되지 않은 문서는 문맥을 반영한 SBERT 문서 임베딩을 얻기 위한 입력으로 사용할 것임.(unpreprocessed_corpus)

전처리 전 문서와 전처리 후 문서를 TopicModelDataPreparation 객체에 넘겨줌. 이 객체는 bag of words와 문맥을 반영한 문서의 BERT 임베딩을 얻게 됨.


여기서 사용할 pretrained BERT는 paraphrase-distilroberta-base-v1임

In [ ]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v1")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

이 결과, tp.vocab으로 단어 집합을 얻을 수 있음. 이는 앞에 vocab에 저장된 단어집합과 동일함.

In [ ]:
len(tp.vocab)